In [2]:
import json
import sys
import pandas as pd
from matplotlib import pyplot as plt
import requests
from keys import client_id, api_key

In [108]:
# SEARCH PARAMETERS
url =  "https://api.yelp.com/v3/businesses/search"
term = 'Indian'
location = 'Washington, D.C.'
categories = "restaurants"
radius = 100

# HEADERS
headers = {'Authorization': 'Bearer {}'.format(api_key),
    }

# URL PARAMETERS
url_params = {"term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
                }

# RESPONSE FROM API SERVER
response = requests.get(url, headers = headers, params = url_params)

In [109]:
def yelp_call(url, url_params, api_key):
#THIS FUNCTION IS CALLED TO MAKE API CALL TO YELP AND RETURN THE RESPONSE

    headers = {'Authorization': 'Bearer {}'.format(api_key),}

    response = requests.get(url, headers = headers, params = url_params)
#    response.status_code
#    response.text

    return response

In [119]:
def parse_data(list_of_data):
#THIS FUNCTION CREATES A TUPLE OF BUSINESS INFORMATION FOR ALL BUSINESSES CAPTURED VIA YELP
# INCLUDES CONVERSION FOR CHANGING "$$" TO NUM FOR PRICE REPRESENTATION
    businesses=[]
    
    for business in list_of_data:
        get_categories = parse_categories(business['categories'])    
        num_categories = len(get_categories)
        if 'price' in business.keys():
            price_val = len(business['price'])
        else:
            price_val = 0
        biz_tuple = (business['id'], business['name'], get_categories, num_categories, price_val, business['review_count'], business['rating'], business['location']['address1'], 
                     business['location']['city'], business['location']['state'], business['location']['zip_code'])
        businesses.append(biz_tuple)

    return businesses

In [120]:
def parse_results(results):
# THIS FUNCTION TAKES RESULTS RETURNED FROM YELP API, PARSES THE DATA, AND STORES AS A DATAFRAME

    #EITHER ONE WORKS
    #data = json.loads(response.text)
    data = results.json()
    num = data['total']
    
    parsed_results = parse_data(data['businesses'])
    df = pd.DataFrame(parsed_results, columns=['business_id', 'name', 'categories', 'num_categories', 'price', 'review_count', 'rating', 'street_address', 'city', 'state', 'zip_code'])
    return df, num

In [121]:
def parse_categories(categories_list):
# THIS FUNCTION TAKES THE LIST OF CATEGORIES FROM A BUSINESS AND CREATES A LIST OF CATEGORIES OF THE BUSINESS

    list_of_categories = []
    x_len = len(categories_list)
    for x in range(0,x_len):
        list_of_categories.append(categories_list[x]['alias'])
    return list_of_categories

In [122]:
def df_save(csv_filepath, parsed_results):
# THIS FUNCTION SAVES A DATAFRAME AS A CSV FILE
    
    f = open(csv_filepath, 'a', encoding="utf-8")
    parsed_results.to_csv(f, header=False)
    f.close()

In [128]:
def get_yelp_reviews(business_id):
# THIS FUNCTION CALLS YELP API TO RETRIEVE ALL REVIEWS FOR A GIVEN BUSINESS ID

    business_reviews = []
    headers = {'Authorization': 'Bearer {}'.format(api_key),}
    url =  "https://api.yelp.com/v3/businesses/{}/reviews".format(business_id)

    review_response = requests.get(url, headers = headers)
    review_data = json.loads(review_response.text)
        
    for review in review_data['reviews']:
        review_tuple = (business_id, review['id'], review['text'], review['rating'], review['time_created'])
        business_reviews.append(review_tuple)

    return business_reviews

In [143]:
def generate_all_reviews(businesses_list):
# THIS FUNCTION IS CALLED TO CREATE A LIST OF REVIEWS FOR ALL BUSINESSES WITHIN ITS PASSED DATA SET

    all_business_reviews = []

    for biz in businesses_list:
        biz_reviews = get_yelp_reviews(biz)
        for each_business in biz_reviews:
            all_business_reviews.append(each_business)

    reviews_results = pd.DataFrame(all_business_reviews, columns = 
                                   ['business_id', 'review_id', 'text', 'rating', 'time_created'])
     
    return reviews_results

In [147]:
def generate_all_yelp_data_la():
# THIS FUNCTION IS THE MAIN DRIVER - SETS SEARCH PARAMETERS, CALLS YELP API, 
# PARSES RETURNED DATA AND STORED IN CSV/DATAFRAME

    term = 'Indian'
    location = 'Los Angeles'
    categories = "restaurants"
    radius = 40000
#   price = 4
    
    url =  "https://api.yelp.com/v3/businesses/search"
    url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "radius" : radius,
                "limit": 50,
#                "price": price,
                }

    all_results_la = pd.DataFrame()
    num = 1
    cur = 0
    
    while (cur < num and cur < 1000):

        url_params['offset'] = cur
        results = yelp_call(url, url_params, api_key)
        parsed_results, num = parse_results(results)
        parsed_reviews = generate_all_reviews(parsed_results.business_id)
        df_save('test_data_la.csv', parsed_results)
        df_save('test_review_la.csv', parsed_reviews)
        all_results_la = all_results_la.append(parsed_results)
        cur += 50

    return all_results_la

In [148]:
def generate_all_yelp_data_dc():
# THIS FUNCTION IS THE MAIN DRIVER - SETS SEARCH PARAMETERS, CALLS YELP API, 
# PARSES RETURNED DATA AND STORED IN CSV/DATAFRAME

    term = 'Indian'
    location = 'Washington, D.C.'
    categories = "restaurants"
    radius = 40000
#   price = 4
    
    url =  "https://api.yelp.com/v3/businesses/search"
    url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "radius" : radius,
                "limit": 50,
#                "price": price,
                }

    all_results_dc = pd.DataFrame()
    num = 1
    cur = 0
    
    while (cur < num and cur < 1000):

        url_params['offset'] = cur
        results = yelp_call(url, url_params, api_key)
        parsed_results, num = parse_results(results)
        parsed_reviews = generate_all_reviews(parsed_results.business_id)
        df_save('test_data_dc.csv', parsed_results)
        df_save('test_review_dc.csv', parsed_reviews)
        all_results_dc = all_results_dc.append(parsed_results)
        cur += 50

    return all_results_dc

In [21]:
all_reviews = pd.read_csv('C:\\Users\\User\\Documents\\Flatiron\\Repository\\flatiron-phase1-yelp\\data\\final_indian_review.csv',
    names=['business_id', 'review_id', 'text', 'rating', 'time_created'], index_col=0)

In [22]:
all_results = pd.read_csv('C:\\Users\\User\\Documents\\Flatiron\\Repository\\flatiron-phase1-yelp\\data\\final_indian_data.csv', converters={'categories': eval}, header=None,
                        names=['business_id', 'name', 'categories', 'num_categories', 'price', 'review_count', 'rating', 'street_address', 'city', 'state'])

In [23]:
def part_4_questions_answers(results_dataframe, reviews_dataframe):
# THIS FUNCTION IS CALLED TO DISPLAY THE PART 4 Q&A RESULTS
    
    most_reviewed_businesses = results_dataframe.sort_values('review_count',ascending=False)[0:5][['name','review_count']]
    highest_rating = results_dataframe['rating'].max()
    high_rated_bus_count = len(results_dataframe.loc[results_dataframe['rating']==highest_rating])#results_dataframe['rating'].max()])
    bus_percent_by_rating = results_dataframe.groupby('rating').count().apply(lambda x: 100*x/x.sum(), axis=0).business_id
    bus_percent_by_price = results_dataframe.groupby('price').count().apply(lambda x: 100*x/x.sum(), axis=0).business_id
    #results_dataframe.sort_values('review_count',ascending=False)[0:1]['business_id'][1]
    most_reviewed_text = all_reviews.loc[all_reviews['business_id']==all_results.sort_values(by='review_count', ascending=False).head(1)['business_id'][1]].text
    max_rated_reviewed_biz = results_dataframe.loc[results_dataframe['rating']==results_dataframe['rating'].max()].sort_values(by='review_count', ascending=False).head(1).values[0][0]
    max_rated_review_text = reviews_dataframe.loc[reviews_dataframe['business_id']==max_rated_reviewed_biz].sort_values(by='time_created', ascending=False).head(1).text
    min_rated_reviewed_biz = results_dataframe.loc[results_dataframe['rating']==results_dataframe['rating'].min()].sort_values(by='review_count', ascending=False).head(1).values[0][0]
    min_rated_review_text = reviews_dataframe.loc[reviews_dataframe['business_id']==min_rated_reviewed_biz].sort_values(by='time_created', ascending=False).head(1).text

    print("1. Top 5 Most reviewed businesses are:")
    print(most_reviewed_businesses)
    print()
    print("2. Number of businesses with highest rating of {}: {}".format(highest_rating, high_rated_bus_count))
    print()
    print("3 & 4. Percent of businesses by Rating: {}".format(bus_percent_by_rating))
    print()
    print("5. Percent of businesses by Price: {}".format(bus_percent_by_price))
    print()
    print("6. Sample reviews of most reviewed business:")
    print(most_reviewed_text)
    print()
    print("7. Latest review of highest rated, highest reviewed business:")
    print(max_rated_review_text)
    print()
    print("8. Latest review of lowest rated, lowest reviewed business:")
    print(min_rated_review_text)


In [25]:
part_4_questions_answers(all_results, all_reviews)

1. Top 5 Most reviewed businesses are:
                     name  review_count
1                  Rasika          3127
13  Amsterdam Falafelshop          1912
2            Kabob Palace          1897
22            Pi Pizzeria          1712
20      True Food Kitchen          1439

2. Number of businesses with highest rating of 5.0: 22

3 & 4. Percent of businesses by Rating: rating
1.0     0.518135
1.5     0.259067
2.0     1.554404
2.5     1.554404
3.0     9.326425
3.5    26.165803
4.0    32.383420
4.5    22.538860
5.0     5.699482
Name: business_id, dtype: float64

5. Percent of businesses by Price: price
0    20.725389
1    16.580311
2    60.880829
3     1.554404
4     0.259067
Name: business_id, dtype: float64

6. Sample reviews of most reviewed business:
3    With 3,000+ reviews already, I don't think I c...
4    I am in DC on Jan 5th for business and staying...
5    My second time at Rasika and it was just as en...
Name: text, dtype: object

7. Latest review of highest rated, highes